## **** Smith-Waterman dynamic programming matrix for local alignment ****

Submit:
*   Ilay Anais 
*   Hadar Pur

Problem 4:

Write a short computer program (in your language of choice) that computes the Smith-Waterman dynamic programming matrix for local alignment of the two sequences:

S = ATAAGGCATTGACCGTATTGCCAA

T = CCCATAGGTGCGGTAGCC


## **** Imports ****


In [ ]:
import numpy as np
import itertools
import math

## **** Global vars ****


In [ ]:
# score function
match = 2
mismatch = -2
gap = -3

S = "ATAAGGCATTGACCGTATTGCCAA"
T = "CCCATAGGTGCGGTAGCC"

## **** Smith-Waterman algorithm alignment ****


In [ ]:
# Local alignment matrix scores
def matrix_scores_local(seq1, seq2, match_score, mismatch_score, gap_score):
  #Create empty matrix A with rows indexed 0..n and columns indexed 0..m
  #Initialize A[i,0] = A[0, j] = 0 and for each i=0..n and j=0.. n
  A = np.zeros((len(seq1)+1, len(seq2)+1), np.int)

  max_score = 0
  max_pos   = None  

  #main loop
  for i, j in itertools.product(range(1, A.shape[0]), range(1, A.shape[1])):
    match   = A[i - 1, j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
    delete  = A[i - 1, j] + gap_score
    insert  = A[i, j - 1] + gap_score

    score = max(match, delete, insert, 0)

    if score > max_score:
      max_score = score
      max_pos = (i,j)

    A[i,j] = score

  assert max_pos is not None

  return A, max_pos, max_score

In [ ]:
# Local alignment traceback alignment
def traceback_matrix_score(A, max_pos, seq1, seq2, s, t):
  i = max_pos[0]
  j = max_pos[1]

  if A[i,j] == 0:
    print("S' = ", seq1)
    print("T' = ", seq2)
    return seq1, seq2

  #left
  if A[i,j-1] == (A[i,j] - gap):
    print(i,j-1)
    seq1 = "-" + seq1
    seq2 = t[j-1] + seq2
    return traceback_matrix_score(A, (i,j-1), seq1, seq2, s, t)

  #up
  if A[i-1,j] == (A[i,j] - gap):
    print(i-1,j)
    seq1 = s[i-1] + seq1
    seq2 = "-" + seq2
    return traceback_matrix_score(A, (i-1,j), seq1, seq2, s, t)

  # up and left
  if A[i-1,j-1] == (A[i,j] - match):
    print(i-1,j-1)
    seq1 = s[i-1] + seq1
    seq2 = t[j-1] + seq2
    return traceback_matrix_score(A, (i-1,j-1), seq1, seq2, s, t)

  # up and left
  if A[i-1,j-1] == (A[i,j] - mismatch):
    print(i-1,j-1)
    seq1 = s[i-1] + seq1
    seq2 = t[j-1] + seq2
    return traceback_matrix_score(A, (i-1,j-1), seq1, seq2, s, t)

  return traceback_matrix_score(A, (i-1, j-1), seq1, seq2, s, t)


In [ ]:
def smith_waterman(seq1, seq2, match_score, mismatch_score, gap_score):
  matrix_scores_results = matrix_scores_local(seq1, seq2, match_score, mismatch_score, gap_score)
  A = matrix_scores_results[0]
  max_pos = matrix_scores_results[1]
  max_score = matrix_scores_results[2] 

  # a. Print the 25x19 matrix of values.
  print("matrix_scores:\n")
  print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in A]))
  print("max_pos = ",max_pos)
  print("max_score = ",max_score)

  # b. Use this matrix to trace back the path representing a max-score alignment.
  # Do not show all 25x19 pointers. Just trace the ones that belong to an optimal
  # alignment. You can do that manually on top of the program output.
  # AND
  # c. Write down the alignment corresponding to the path you traced in (b).
  print("\n\nLets traceback:")
  traceback_matrix_score(A, max_pos, "", "", S, T)
  print("max_pos = ", max_pos)

  # d. Notice that there are several optimal local alignments of S, T. Specify another
  # optimal local alignment that does not overlap the alignment you specified in
  # (c) above.
  # Lets observe the matrix and will fins another max_pos
  max_pos2 = (22,18)
  print("\n\nLets traceback again:")
  traceback_matrix_score(A, max_pos2, "", "", S, T)
  print("max_pos2 = ", max_pos2)
  return


smith_waterman(S, T, match, mismatch, gap)

matrix_scores:

   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0  -2  -2  -2  -2   2  -1  -2  -2   2  -1  -2  -2  -2   2  -1  -2  -2  -2
   0  -2  -4  -4  -4   0   0  -3  -4   0   0  -3  -4  -4   0   0  -3  -4  -4
   0  -2  -4  -6  -6  -2  -2  -2  -5  -2  -2  -2  -5  -6  -2  -2  -2  -5  -6
   0  -2  -4  -6  -8  -4  -4  -4  -4  -3  -4  -4  -4  -7  -4  -4  -4  -4  -7
   0  -2  -4  -6  -8  -6  -6  -6  -6  -2  -5  -6  -6  -6  -5  -6  -6  -6  -6
   0  -2  -4  -6  -8  -6  -8  -8  -8  -4  -4  -7  -8  -8  -4  -7  -8  -8  -8
   0  -2  -4  -6  -8  -6  -8 -10 -10  -6  -6  -6  -9 -10  -6  -6  -9 -10 -10
   0  -2  -4  -6  -8  -6  -8 -10 -12  -8  -8  -8  -8 -11  -8  -8  -8 -11 -12
   0  -2  -4  -6  -8  -6  -8 -10 -12 -10 -10 -10 -10 -10  -9 -10 -10 -10 -13
   0  -2  -4  -6  -8  -6  -8 -10 -12 -10 -12 -12 -12 -12  -8 -11 -12 -12 -12
   0  -2  -4  -6  -8  -6  -8 -10 -12 -10 -12 -14 -14 -14 -10 -10 -13 -14 -14
   0  -2  -4  -6  -8  -6  -8 -10 -12 -10 -12 -14 -16 -16 -12

IndexError: ignored

## **** Hircshberg algorithm for local alignment ****


In [ ]:
# Modify your code to compute a maximum score local alignment in linear
# space complexity (O(n)) using Hirschberg’s technique, and your algorithm from Problem 2. 
# Minimize the space complexity of your implementation should be linear in the length of the shorter sequence (n), 
# and the time complexity of the implementation should be O(mn), as guaranteed by Hitrschberg’s technique..
def hirschberg_scores(seq1, seq2, match_score, mismatch_score, gap_score):
  #Create empty arrays A and B with cells indexed minimum of n or m
  #Initialize A and B to 0 values
  A = np.zeros(min(len(seq1)+1, len(seq2)+1), np.int)
  B = np.zeros(min(len(seq1)+1, len(seq2)+1), np.int)

  #pointer for highest score reached and it's position
  max_score = 0
  max_pos   = (len(seq1),len(seq2))  

  #we make sure seq2 is the shorter one for looping purposes
  if len(seq1) < len(seq2):
    temp = seq1
    seq1 = seq2
    seq2 = temp

  print(' '.join(['{:4}'.format(item) for item in A]))

  #main loop O(mn) due to each step taking constant time and going over all i and j is lengths of each sequence + 1
  for i, j in itertools.product(range(1, len(seq1)+1), range(1, len(seq2)+1)):
    match = A[j-1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
    delete  = B[j-1] + gap_score
    insert  = A[j] + gap_score

    score = max(match, delete, insert, 0)

    if score > max_score:
      max_score = score
      max_pos = (i,j)

    if score == max_score and i+j < max_pos[0]+max_pos[1]:
      max_pos = (i, j)
      
    B[j] = score

    #if we reach end of row
    if(j == len(seq2)):
      #Copy B to A so that we only use two rows at each point
      A = B.copy()
      print(' '.join(['{:4}'.format(item) for item in A]))

  return A, max_pos, max_score

In [ ]:
#We cut the sequences short to the maximum position
def sequence_cut_flip(seq1, seq2, max_pos):
  if len(seq1) < len(seq2):
    temp = seq1
    seq1 = seq2
    seq2 = temp

  seq1 = seq1[0:max_pos[0]]
  seq2 = seq2[0:max_pos[1]]

  seq1 = seq1[::-1]
  seq2 = seq2[::-1]

  return seq1, seq2

In [ ]:
#We cut the sequences short to the maximum position
def sequence_split(seq1, seq2, pos):
  if len(seq1) < len(seq2):
    temp = seq1
    seq1 = seq2
    seq2 = temp

  print("sequence_cut before cutting:")
  print("seq1 = ", seq1)
  print("seq2 = ", seq2)

  seq1_1, seq1_2 = seq1[:pos[0]], seq1[pos[0]:]

  seq2_1, seq2_2 = seq2[:pos[1]], seq2[pos[1]:]

  print("seq1_1 = ", seq1_1)
  print("seq1_2 = ", seq1_2)
  print("seq2_1 = ", seq2_1)
  print("seq2_2 = ", seq2_2)

  return seq1_1, seq1_2, seq2_1, seq2_2

In [ ]:
#We cut the sequences short to the maximum position
def sequence_flip(seq1, seq2):
  if len(seq1) < len(seq2):
    temp = seq1
    seq1 = seq2
    seq2 = temp

  seq1 = seq1[::-1]
  seq2 = seq2[::-1]

  return seq1, seq2

In [ ]:
def hirschberg_global_scores(seq1, seq2, match_score, mismatch_score, gap_score):
  #Create empty arrays A and B with cells indexed minimum of n or m
  #Initialize A and B to 0 values
  A = np.zeros(min(len(seq1)+1, len(seq2)+1), np.int)
  B = np.zeros(min(len(seq1)+1, len(seq2)+1), np.int)

  #pointer for highest score reached and it's position
  max_score     = 0
  max_pos       = None  
  middle_pos    = None  

  #we make sure seq2 is the shorter one for looping purposes
  if len(seq1) < len(seq2):
    temp = seq1
    seq1 = seq2
    seq2 = temp

  print(' '.join(['{:4}'.format(item) for item in A]))

  #main loop O(mn) due to each step taking constant time and going over all i and j is lengths of each sequence + 1
  for i, j in itertools.product(range(1, len(seq1)+1), range(1, len(seq2)+1)):
    match = A[j-1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
    delete  = B[j-1] + gap_score
    insert  = A[j] + gap_score

    score = max(match, delete, insert)

    if score > max_score:
      max_score = score
      max_pos = (i,j)

    B[j] = score

    # got the middle section of the sub seq
    if(i == math.floor(len(seq1)/2) and j == math.ceil(len(seq2)/2) ):
      middle_pos = (i, j)
      
    #if we reach end of row
    if(j == len(seq2)):
      #Copy B to A so that we only use two rows at each point
      A = B.copy()
      print(' '.join(['{:4}'.format(item) for item in A]))

  assert middle_pos is not None

  return A, max_pos, middle_pos, max_score

In [ ]:
# Global alignment matrix scores
def matrix_scores_global(seq1, seq2, match_score, mismatch_score, gap_score):
  #Create empty matrix A with rows indexed 0..n and columns indexed 0..m
  #Initialize A[i,0] = A[0, j] = 0 and for each i=0..n and j=0.. n
  A = np.zeros((len(seq1)+1, len(seq2)+1), np.int)

  max_score = 0
  max_pos   = None  

  #main loop
  for i, j in itertools.product(range(1, A.shape[0]), range(1, A.shape[1])):
    match   = A[i - 1, j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
    delete  = A[i - 1, j] + gap_score
    insert  = A[i, j - 1] + gap_score

    score = max(match, delete, insert)

    if score > max_score:
      max_score = score

    A[i,j] = score
    max_pos = (i,j)

  assert max_pos is not None

  return A, max_pos, max_score

In [ ]:
# Global alignment traceback alignment
def traceback_matrix_score_global(A, max_pos, seq1, seq2, s, t):
    i = max_pos[0]
    j = max_pos[1]

    if i == 0 and j == 0:
        print("S' = ", seq1)
        print("T' = ", seq2)
        return seq1, seq2

    if i == 0:
        print(i, j - 1)
        seq1 = "-" + seq1
        seq2 = t[j - 1] + seq2
        return traceback_matrix_score_global(A, (i, j - 1), seq1, seq2, s, t)

    if j == 0:
        print(i - 1, j)
        seq1 = s[i - 1] + seq1
        seq2 = "-" + seq2
        return traceback_matrix_score_global(A, (i-1, j), seq1, seq2, s, t)

    # left
    if A[i, j - 1] == (A[i, j] - gap):
        print(i, j - 1)
        seq1 = "-" + seq1
        seq2 = t[j - 1] + seq2
        return traceback_matrix_score_global(A, (i, j - 1), seq1, seq2, s, t)

    # up
    if A[i - 1, j] == (A[i, j] - gap):
        print(i - 1, j)
        seq1 = s[i - 1] + seq1
        seq2 = "-" + seq2
        return traceback_matrix_score_global(A, (i - 1, j), seq1, seq2, s, t)

    # up and left
    if A[i - 1, j - 1] == (A[i, j] - match):
        print(i - 1, j - 1)
        seq1 = s[i - 1] + seq1
        seq2 = t[j - 1] + seq2
        return traceback_matrix_score_global(A, (i - 1, j - 1), seq1, seq2, s, t)

    # up and left
    if A[i - 1, j - 1] == (A[i, j] - mismatch):
        print(i - 1, j - 1)
        seq1 = s[i - 1] + seq1
        seq2 = t[j - 1] + seq2
        return traceback_matrix_score_global(A, (i - 1, j - 1), seq1, seq2, s, t)

    return traceback_matrix_score_global(A, (i - 1, j - 1), seq1, seq2, s, t)


In [ ]:
def hircshberg_smith_waterman(seq1, seq2, match_score, mismatch_score, gap_score):
    # lets run hircshberg algorithm for local alignment on S and T
    print("\nhirschberg_scores:")
    hirschberg_results = hirschberg_scores(seq1, seq2, match_score, mismatch_score, gap_score)
    A = hirschberg_results[0]
    max_pos = hirschberg_results[1]
    max_score = hirschberg_results[2]

    print("max_pos = ", max_pos)
    print("max_score = ", max_score)

    # now lets cut and revers the the S and T
    print("\nLets cut and flip the 2 sequences S and T (len(S) < len(T) always):")
    sequence_cut_flip_results = sequence_cut_flip(seq1, seq2, max_pos)

    # the sub sequences seq1, seq2
    sub_seq1 = sequence_cut_flip_results[0]
    sub_seq2 = sequence_cut_flip_results[1]
    print("sub_seq1 = ", sub_seq1)
    print("sub_seq2 = ", sub_seq2)

    # lets run hircshberg algorithm for local alignment on sub_seq1 and sub_seq2
    print("\nhirschberg_scores after cut & flip:")
    hirschberg_results = hirschberg_scores(sub_seq1, sub_seq2, match_score, mismatch_score, gap_score)
    A = hirschberg_results[0]
    max_pos = hirschberg_results[1]
    max_score = hirschberg_results[2]
    print("max_pos = ", max_pos)
    print("max_score = ", max_score)

    print("\nLets cut and flip the 2 sub-sequences sub_seq1 and sub_seq2 (len(seq1) < len(seq2) always):")
    # now lets cut and revers the the sub_seq1 and sub_seq2
    sequence_cut_flip_results = sequence_cut_flip(sub_seq1, sub_seq2, max_pos)

    # the sub sequences seq1, seq2
    sub_seq1 = sequence_cut_flip_results[0]
    sub_seq2 = sequence_cut_flip_results[1]
    print("sub_seq1 = ", sub_seq1)
    print("sub_seq2 = ", sub_seq2)

    # lets run hircshberg algorithm for global alignment on sub_seq1 and sub_seq2
    print("\nhirschberg_scores of sub-sequences sub_seq1 and sub_seq2 for global (len(seq1) < len(seq2) always):")
    hirsberg_global_results = hirschberg_global_scores(sub_seq1, sub_seq2, match_score, mismatch_score, gap_score)
    A = hirsberg_global_results[0]
    max_pos = hirsberg_global_results[1]
    middle_pos = hirsberg_global_results[2]
    max_score = hirsberg_global_results[3]

    print("max_pos = ", max_pos)
    print("middle_pos = ", middle_pos)
    print("max_score = ", max_score)

    # Lets cut the both sequences to an half
    print("\nLets cut the both sequences to an half (len(seq1) < len(seq2) always):")
    sequences = sequence_split(sub_seq1, sub_seq2, middle_pos)
    seq1_1 = sequences[0]
    seq1_2 = sequences[1]
    seq2_1 = sequences[2]
    seq2_2 = sequences[3]

    print("\nLets run global alignment function for both halves")
    # revers first
    seq1_1, seq2_1 = sequence_flip(seq1_1, seq2_1)
    # Lets run global alignment function half 1
    matrix_scores_results_half1 = matrix_scores_global(seq1_1, seq2_1, match, mismatch, gap)
    A1 = matrix_scores_results_half1[0]
    max_pos1 = matrix_scores_results_half1[1]
    max_score1 = matrix_scores_results_half1[2]

    print("\nmatrix_scores half1:\n")
    print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in A1]))
    print("max_pos1 = ", max_pos1)
    print("max_score1 = ", max_score1)

    # revers first
    seq1_2, seq2_2 = sequence_flip(seq1_2, seq2_2)
    # Lets run global alignment function half 2
    matrix_scores_results_half2 = matrix_scores_global(seq1_2, seq2_2, match, mismatch, gap)
    A2 = matrix_scores_results_half2[0]
    max_pos2 = matrix_scores_results_half2[1]
    max_score2 = matrix_scores_results_half2[2]

    print("\nmatrix_scores half2:\n")
    print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in A2]))
    print("max_pos2 = ", max_pos2)
    print("max_score2 = ", max_score2)

    print("\n\nLets traceback for half 1:")
    s1, t1 = traceback_matrix_score_global(A1, max_pos1, "", "", seq1_1, seq2_1)
    s1, t1 = sequence_flip(s1, t1)

    print("\n\nLets traceback for half 2:")
    s2, t2 = traceback_matrix_score_global(A2, max_pos2, "", "", seq1_2, seq2_2)
    s2, t2 = sequence_flip(s2, t2)

    print("\n\nFinal alignment:")
    print("S' = ", s1 + s2)
    print("T' = ", t1 + t2)

    return

hircshberg_smith_waterman(S, T, match, mismatch, gap)


hirschberg_scores:
   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
   0    0    0    0    2    0    2    0    0    0    0    0    0    0    0    2    0    0    0
   0    0    0    0    0    4    1    0    0    2    0    0    0    0    2    0    0    0    0
   0    0    0    0    2    1    6    3    0    0    0    0    0    0    0    4    1    0    0
   0    0    0    0    2    0    3    4    1    0    0    0    0    0    0    2    2    0    0
   0    0    0    0    0    0    0    5    6    3    2    0    2    2    0    0    4    1    0
   0    0    0    0    0    0    0    2    7    4    5    2    2    4    1    0    2    2    0
   0    2    2    2    0    0    0    0    4    5    2    7    4    1    2    0    0    4    4
   0    0    0    0    4    1    2    0    1    2    3    4    5    2    0    4    1    1    2
   0    0    0    0    1    6    3    0    0    3    0    1    2    3    4    1    2    0    0
   0    0    0    0    0    3 